### **Importações + Tratamento das bases de dados**

In [68]:
import pandas as pd 
import matplotlib.pyplot as plt

In [69]:
cadastro_funcionarios = pd.read_csv(r'data/CadastroFuncionarios.csv', sep=';', decimal=',') 
# decimal=',' - Modificar para sisema numérico Py
display(cadastro_funcionarios.head())

,ID Funcionário,Estado Civil,Nome Completo,Salario Base,Impostos,Beneficios,VT,VR,Cargo,Area
0,1,C,Gabriel Mesquita,21910,10955.0,4382.0,242,719.04,Diretor,Operações
1,2,C,João Haddad,5404,2702.0,1080.8,154,574.56,Estagiário,Logística
2,3,C,Amanda Marques Ribeiro,16066,8033.0,3213.2,154,729.12,Estagiário,Administrativo
3,4,C,Guilherme Nunez,21305,10652.5,4261.0,220,524.16,Analista,Administrativo
4,5,C,Adelino Gomes,5098,2549.0,1019.6,176,725.76,Analista,Administrativo


In [70]:
cadastro_clientes =  pd.read_csv(r'data/CadastroClientes.csv', sep=';', decimal=',')
display(cadastro_clientes.head())

,ID Cliente,Cliente,Valor Contrato Mensal
0,1,Teixeira Gonçalves,540
1,2,Souza Santos,1260
2,3,Emídio Alves,3195
3,4,Santos Costa,2520
4,5,Do Monteiro,3510


In [71]:
servicos = pd.read_excel(r'data/BaseServiçosPrestados.xlsx')
display(servicos.head())

,Codigo do Servico,ID Funcionário,ID Cliente,Tempo Total de Contrato (Meses)
0,OS0001,67,1,14
1,OS0002,17,2,12
2,OS0003,116,4,14
3,OS0004,37,5,8
4,OS0005,130,6,8


------
## **Desafio 1**
Estimar o total de gasto que a empresa tem com os funcionários

In [72]:
cadastro_funcionarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID Funcionário  114 non-null    int64  
 1   Estado Civil    114 non-null    object 
 2   Nome Completo   114 non-null    object 
 3   Salario Base    114 non-null    int64  
 4   Impostos        114 non-null    float64
 5   Beneficios      114 non-null    float64
 6   VT              114 non-null    int64  
 7   VR              114 non-null    float64
 8   Cargo           114 non-null    object 
 9   Area            114 non-null    object 
dtypes: float64(3), int64(3), object(4)
memory usage: 9.0+ KB


In [73]:
cadastro_funcionarios['Gasto por Funcionário'] = (cadastro_funcionarios['Salario Base'] +
                                                  cadastro_funcionarios['Impostos'] +
                                                  cadastro_funcionarios['Beneficios'] +
                                                  cadastro_funcionarios['VT'] +
                                                  cadastro_funcionarios['VR'])
gastos_totais = sum(cadastro_funcionarios['Gasto por Funcionário'])

In [74]:
print('Gasto total com funcionários R$ {:,}'.format(gastos_totais))

Gasto total com funcionários R$ 2,717,493.22


## **Desafio 2**

Faturamento total da empresa

In [75]:
cliente_venda = cadastro_clientes.merge(servicos, on='ID Cliente')
display(cliente_venda)

,ID Cliente,Cliente,Valor Contrato Mensal,Codigo do Servico,ID Funcionário,Tempo Total de Contrato (Meses)
0,1,Teixeira Gonçalves,540,OS0001,67,14
1,2,Souza Santos,1260,OS0002,17,12
2,4,Santos Costa,2520,OS0003,116,14
3,5,Do Monteiro,3510,OS0004,37,8
4,6,Soares Lobo,2340,OS0005,130,8
...,...,...,...,...,...,...
232,315,Americo Bomfim,1575,OS0233,111,4
233,316,Manoel Costa,3690,OS0234,124,8
234,317,Gomes Machado,2385,OS0235,72,6
235,319,Pereira Fazenda,4185,OS0236,90,14


In [76]:
cliente_venda['Faturamento Empresa'] = cliente_venda['Valor Contrato Mensal'] * cliente_venda['Tempo Total de Contrato (Meses)']
faturamento_total = sum(cliente_venda['Faturamento Empresa'])

In [77]:
print('Faturamento total R${:,}'.format(faturamento_total))

Faturamento total R$5,519,160


## **Desafio 3**

Descobrir a porcentagem de funcionárioos que fecharam algum contrato

In [78]:
qntd_funcionarios_venderam = cliente_venda['ID Funcionário'].tolist()
qntd_funcionarios_venderam = set(qntd_funcionarios_venderam)

In [79]:
porcentagem_vendas_funcionario = round(len(qntd_funcionarios_venderam) / len(cadastro_funcionarios), 2) * 100

In [80]:
print(f'Porcentagem de funcionários que fecharam contrato = {porcentagem_vendas_funcionario}%')

Porcentagem de funcionários que fecharam contrato = 87.0%


## **Desafio 4**

Descobrir a quantidade de contratos por área

In [81]:
funcionarios_area = cliente_venda[['ID Funcionário']].merge(cadastro_funcionarios[['ID Funcionário', 'Area']])
display(funcionarios_area.head())

,ID Funcionário,Area
0,67,Financeiro
1,67,Financeiro
2,67,Financeiro
3,67,Financeiro
4,17,Logística


In [82]:
qnt_fun_per_area = funcionarios_area['Area'].value_counts()
dic_funcionarios_area = qnt_fun_per_area.to_dict()

In [83]:
fig = plt.figure(figsize=(10,5), dpi=1800)
plt.grid(linestyle='--')
plt.bar(dic_funcionarios_area.keys(), dic_funcionarios_area.values(), color='blue')
plt.xticks(rotation=90)

([0, 1, 2, 3, 4],
 [Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, ''),
  Text(0, 0, '')])

## **Desafio 5**

Descobrir a quantidade de clientes por área

In [92]:
novo_cad_funcionarios = cadastro_funcionarios.set_index('Area')
display(novo_cad_funcionarios.head())

,ID Funcionário,Estado Civil,Nome Completo,Salario Base,Impostos,Beneficios,VT,VR,Cargo,Gasto por Funcionário
Area,,,,,,,,,,
Operações,1,C,Gabriel Mesquita,21910,10955.0,4382.0,242,719.04,Diretor,38208.04
Logística,2,C,João Haddad,5404,2702.0,1080.8,154,574.56,Estagiário,9915.36
Administrativo,3,C,Amanda Marques Ribeiro,16066,8033.0,3213.2,154,729.12,Estagiário,28195.32
Administrativo,4,C,Guilherme Nunez,21305,10652.5,4261.0,220,524.16,Analista,36962.66
Administrativo,5,C,Adelino Gomes,5098,2549.0,1019.6,176,725.76,Analista,9568.36


In [93]:
funcionarios_area = novo_cad_funcionarios.groupby('Area').count()
funcionarios_area = funcionarios_area[['ID Funcionário']]

In [94]:
display(funcionarios_area)

,ID Funcionário
Area,
Administrativo,26
Comercial,26
Financeiro,18
Logística,21
Operações,23


**Funcionamento do groupby**

- Passo 1:
Existirá uma coluna de referência = Aquela que sepera as informações 

***PS:*** No caso do desafio 5, queriamos saber a quantidade de funcionários **por área**, logo área pe a coluna de referência.

- Passo 2:
Tornar a coluna de referência index. 

- Passo 3: 
    
        variavel = DataFrame.gropby('index').operação()
        variavel = variavel[['Nome das colunas que queremos saber a referência']]

## **Desafio 6**

Faturamento médio mensal

In [95]:
faturamento_medio_mensal = cliente_venda['Valor Contrato Mensal'].mean()

In [98]:
print('Faturamento médio mensal = R$ {:,.4f}'.format(faturamento_medio_mensal))

Faturamento médio mensal = R$ 2,438.3544
